### Data sets were acquired from https://www.kaggle.com/c/titanic

In [1]:
import pandas as pd
import numpy as np
import sidetable
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

df_train = pd.read_csv('train.csv', index_col=['PassengerId'])
test = pd.read_csv('test.csv', index_col=['PassengerId'])

In [2]:
# Have a look at the first five rows
df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_train.shape

(891, 11)

In [4]:
def drop_irrelevant(df):
    return df.drop(columns=['Name', 'Ticket', 'Cabin'])
df_train = drop_irrelevant(df_train)
X_test = drop_irrelevant(test)

In [5]:
# Get X_train and y_train
y = df_train.Survived.copy()
X = df_train.drop(columns='Survived').copy()

In [6]:
# Retrive num_cols and cat_cols
num_cols = list(X._get_numeric_data().columns)
cat_cols = list(set(X.columns) - set(num_cols))

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector

cat_pipe = make_pipeline(
            (SimpleImputer(strategy='most_frequent')),
            (OneHotEncoder(drop='first', handle_unknown='error'))
            )
num_pipe = make_pipeline(
            (IterativeImputer()),
            (StandardScaler())
            )            
preprocess_pipeline = make_column_transformer(
            (cat_pipe, cat_cols),
            (num_pipe, num_cols)
            )

In [8]:
preprocessor =  preprocess_pipeline.fit(X)
X = preprocessor.transform(X)
X_test = preprocessor.transform(X_test)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, stratify=y)

In [10]:
'''
Credit source: 
    https://www.tensorflow.org/tutorials/keras/keras_tuner
    https://github.com/keras-team/keras-tuner/blob/master/examples/cifar10.py
'''

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Create the keras tuner model.
def build_model(hp):
    hp_drop_out = hp.Float('dropout', 0, 0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))
#         model.add(BatchNormalization())
        model.add(Dropout(hp_drop_out))
#     model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

import kerastuner as kt

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy', 
                     max_epochs=16,
                     overwrite=False)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [11]:
es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=10, min_delta=1e-3)
tuner.search(X_train, y_train, epochs=64, batch_size=32, validation_data=(X_val, y_val), callbacks=[es])

INFO:tensorflow:Oracle triggered exit


In [12]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=64, validation_data=(X_val, y_val))

Epoch 1/64
24/24 [==============================] - 2s 34ms/step - loss: 0.6712 - accuracy: 0.6190 - val_loss: 0.5770 - val_accuracy: 0.6194
Epoch 2/64
24/24 [==============================] - 0s 10ms/step - loss: 0.5895 - accuracy: 0.6114 - val_loss: 0.5567 - val_accuracy: 0.7761
Epoch 3/64
24/24 [==============================] - 0s 10ms/step - loss: 0.5552 - accuracy: 0.8017 - val_loss: 0.5214 - val_accuracy: 0.8209
Epoch 4/64
24/24 [==============================] - 0s 11ms/step - loss: 0.5224 - accuracy: 0.8354 - val_loss: 0.5188 - val_accuracy: 0.7910
Epoch 5/64
24/24 [==============================] - 0s 11ms/step - loss: 0.5095 - accuracy: 0.8031 - val_loss: 0.4519 - val_accuracy: 0.8284
Epoch 6/64
24/24 [==============================] - 0s 10ms/step - loss: 0.5271 - accuracy: 0.8071 - val_loss: 0.4231 - val_accuracy: 0.8284
Epoch 7/64
24/24 [==============================] - 0s 10ms/step - loss: 0.4572 - accuracy: 0.8184 - val_loss: 0.4061 - val_accuracy: 0.8657
Epoch 8/64
24

In [13]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 62


In [14]:
# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)

Epoch 1/62
24/24 [==============================] - 2s 9ms/step - loss: 0.6678 - accuracy: 0.5749
Epoch 2/62
24/24 [==============================] - 0s 8ms/step - loss: 0.5899 - accuracy: 0.6700
Epoch 3/62
24/24 [==============================] - 0s 8ms/step - loss: 0.6398 - accuracy: 0.6934
Epoch 4/62
24/24 [==============================] - 0s 9ms/step - loss: 0.5214 - accuracy: 0.8204
Epoch 5/62
24/24 [==============================] - 0s 8ms/step - loss: 0.5406 - accuracy: 0.7825
Epoch 6/62
24/24 [==============================] - 0s 9ms/step - loss: 0.4838 - accuracy: 0.8087
Epoch 7/62
24/24 [==============================] - 0s 8ms/step - loss: 0.4622 - accuracy: 0.8299
Epoch 8/62
24/24 [==============================] - 0s 8ms/step - loss: 0.4665 - accuracy: 0.8109
Epoch 9/62
24/24 [==============================] - 0s 9ms/step - loss: 0.4378 - accuracy: 0.8173
Epoch 10/62
24/24 [==============================] - 0s 8ms/step - loss: 0.4549 - accuracy: 0.8067
Epoch 11/62
24/24 [

In [15]:
eval_result = hypermodel.evaluate(X_val, y_val)
print("[test loss, test accuracy]:", eval_result)

5/5 [==============================] - 0s 2ms/step - loss: 0.4745 - accuracy: 0.8731
[test loss, test accuracy]: [0.47452569007873535, 0.8731343150138855]


In [16]:
# Retrain the model
hypermodel.fit(X, y, epochs=best_epoch)

Epoch 1/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3853 - accuracy: 0.8474
Epoch 2/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3764 - accuracy: 0.8519
Epoch 3/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3820 - accuracy: 0.8541
Epoch 4/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3782 - accuracy: 0.8642
Epoch 5/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3551 - accuracy: 0.8563
Epoch 6/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3445 - accuracy: 0.8709
Epoch 7/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3541 - accuracy: 0.8642
Epoch 8/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3788 - accuracy: 0.8530
Epoch 9/62
28/28 [==============================] - 0s 9ms/step - loss: 0.3717 - accuracy: 0.8620
Epoch 10/62
28/28 [==============================] - 0s 8ms/step - loss: 0.3776 - accuracy: 0.8575
Epoch 11/62
28/28 [

In [17]:
# from keras.models import save_model
# hypermodel.save('best_model.h5')

In [18]:
# Load the model and predict
# from keras.models import load_model
# hypermodel = load_model('best_model.h5')
y_pred = hypermodel.predict(X_test)
y_pred = y_pred.astype(int)

In [19]:
y_pred = pd.DataFrame(y_pred, columns=['Survived'], index=test.index)
# saving the dataframe 
y_pred.to_csv('Predictions.csv') 

In [20]:
import winsound
duration = 100  # milliseconds
freq = 3000  # Hz
winsound.Beep(freq, duration)